In [1]:
# Cell 2: Import libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime

print("\n Libraries imported")

# PURPOSE: Final recommendations based on economic analysis

# Setup and load all variables from Q3
print("=" * 60)
print("LOADING ALL VARIABLES FROM Q3")
print("=" * 60)

# Load the economic dictionary from Q3
%store -r economic_data

# Also load individual variables if available
%store -r payback_year
%store -r roi_20_years
%store -r optimal_scenario
%store -r estimated_annual_kwh
%store -r february_total_kwh

# Check if loaded successfully
if 'economic_data' in dir() and economic_data is not None:
    print(f" Loaded economic_data dictionary with {len(economic_data)} items")
    
    # Extract key variables
    panel_params = economic_data.get('panel_params', {})
    econ_params = economic_data.get('econ_params', {})
    panel_power = economic_data.get('panel_power', 3.0)
    installation_cost = economic_data.get('installation_cost', 0)
    
    february_total_kwh = economic_data.get('february_total_kwh', 0)
    avg_daily_kwh = economic_data.get('avg_daily_kwh', 0)
    estimated_annual_kwh = economic_data.get('estimated_annual_kwh', 0)
    
    optimal_scenario = economic_data.get('optimal_scenario', {})
    optimal_self_consumption_pct = economic_data.get('optimal_self_consumption_pct', 70)
    optimal_feb_benefit = economic_data.get('optimal_feb_benefit', 0)
    estimated_annual_benefit = economic_data.get('estimated_annual_benefit', 0)
    
    payback_years = economic_data.get('payback_years', 999)
    payback_years_display = economic_data.get('payback_years_display', 'Unknown')
    roi_20_years = economic_data.get('roi_20_years', 0)
    total_profit_20_years = economic_data.get('total_profit_20_years', 0)
    
    grid_cost_20_years = economic_data.get('grid_cost_20_years', 0)
    solar_savings_20_years = economic_data.get('solar_savings_20_years', 0)
    net_benefit_vs_grid = economic_data.get('net_benefit_vs_grid', 0)
    break_even_rate = economic_data.get('break_even_rate', 0)
    coverage_percentage = economic_data.get('coverage_percentage', 0)
    
    optimal_panel_size = economic_data.get('optimal_panel_size', panel_power)
    
    print("\n Key metrics loaded:")
    print(f"   Panel power: {panel_power} kWp")
    print(f"   February production: {february_total_kwh:.2f} kWh")
    print(f"   Estimated annual: {estimated_annual_kwh:.0f} kWh/year")
    print(f"   Payback period: {payback_years_display}")
    print(f"   ROI (20 years): {roi_20_years:.0f}%")
    
else:
    print(" Failed to load economic_data from Q3")
    print("Please run Q3 notebook first")
    raise ValueError("Cannot continue without data from Q3")


 Libraries imported
LOADING ALL VARIABLES FROM Q3
no stored variable or alias payback_year
 Loaded economic_data dictionary with 33 items

 Key metrics loaded:
   Panel power: 3.0 kWp
   February production: 280.00 kWh
   Estimated annual: 441 kWh/year
   Payback period: Never pays back
   ROI (20 years): -137%


In [2]:
# Investment Summary
print("=" * 60)
print("INVESTMENT SUMMARY")
print("=" * 60)

print(f"""
PANEL CONFIGURATION:
   Panel power: {panel_power} kWp
   Installation cost: €{installation_cost:,.0f}
   Optimal self-consumption: {optimal_self_consumption_pct}%

PRODUCTION:
   February production: {february_total_kwh:.2f} kWh
   Average daily (Feb): {avg_daily_kwh:.2f} kWh/day
   Estimated annual: {estimated_annual_kwh:.0f} kWh/year
   Coverage of consumption: {coverage_percentage:.1f}%

FINANCIAL:
   February benefit: €{optimal_feb_benefit:.2f}
   Estimated annual benefit: €{estimated_annual_benefit:.0f}/year
   Payback period: {payback_years_display}
   20-year profit: €{total_profit_20_years:,.0f}
   ROI after 20 years: {roi_20_years:.0f}%
""")

INVESTMENT SUMMARY

PANEL CONFIGURATION:
   Panel power: 3.0 kWp
   Installation cost: €5,400
   Optimal self-consumption: 70.0%

PRODUCTION:
   February production: 280.00 kWh
   Average daily (Feb): 10.00 kWh/day
   Estimated annual: 441 kWh/year
   Coverage of consumption: 124.4%

FINANCIAL:
   February benefit: €77.28
   Estimated annual benefit: €-28/year
   Payback period: Never pays back
   20-year profit: €-7,383
   ROI after 20 years: -137%



In [3]:
# Cell 4: Comparison with Grid
print("=" * 60)
print("SOLAR VS GRID COMPARISON")
print("=" * 60)

annual_grid_cost = econ_params.get('household_consumption', 2700) * econ_params.get('electricity_rate', 0.30)

print(f"""
WITHOUT SOLAR (all from grid):
   Annual electricity cost: €{annual_grid_cost:.0f}
   Cost over 20 years: €{grid_cost_20_years:,.0f}

WITH SOLAR:
   Total savings over 20 years: €{solar_savings_20_years:,.0f}
   Net benefit vs grid: €{net_benefit_vs_grid:,.0f}

BREAK-EVEN ANALYSIS:
   Current electricity rate: €{econ_params.get('electricity_rate', 0.30)}/kWh
   Break-even rate: €{break_even_rate:.2f}/kWh
   {'Solar is cheaper' if break_even_rate < econ_params.get('electricity_rate', 0.30) else 'Grid is cheaper'} at current rates
""")

SOLAR VS GRID COMPARISON

WITHOUT SOLAR (all from grid):
   Annual electricity cost: €810
   Cost over 20 years: €22,418

WITH SOLAR:
   Total savings over 20 years: €-1,983
   Net benefit vs grid: €-24,401

BREAK-EVEN ANALYSIS:
   Current electricity rate: €0.3/kWh
   Break-even rate: €0.71/kWh
   Grid is cheaper at current rates



In [5]:
# Sensitivity Analysis Results
print("=" * 60)
print("OPTIMAL PANEL SIZE ANALYSIS")
print("=" * 60)

print(f"""
YOUR CURRENT SIZE: {panel_power} kWp
   Payback period: {payback_years_display}

RECOMMENDED SIZE: {optimal_panel_size} kWp
   {' Larger system would pay back faster' if optimal_panel_size > panel_power else ' Current size is optimal'}
   {' Consider upgrading' if optimal_panel_size > panel_power + 0.5 else ''}
""")

# Show sensitivity table if available
sensitivity_results = economic_data.get('sensitivity_results')
if sensitivity_results is not None and isinstance(sensitivity_results, pd.DataFrame):
    print("\nSENSITIVITY ANALYSIS - PANEL SIZES:")
    print(sensitivity_results.to_string(index=False))

OPTIMAL PANEL SIZE ANALYSIS

YOUR CURRENT SIZE: 3.0 kWp
   Payback period: Never pays back

RECOMMENDED SIZE: 6.0 kWp
    Larger system would pay back faster
    Consider upgrading


SENSITIVITY ANALYSIS - PANEL SIZES:
 panel_size_kw  feb_production_kwh  annual_production_kwh  installation_cost_euro  annual_benefit_euro  payback_years
           2.0               186.7                    294                    3600                  -68            inf
           2.5               233.3                    367                    4500                  -48            inf
           3.0               280.0                    441                    5400                  -28            inf
           3.5               326.7                    514                    6300                   -7            inf
           4.0               373.3                    588                    7200                   12          585.9
           4.5               420.0                    661                

In [6]:
# Cell 6: SCORING SYSTEM
print("=" * 60)
print("INVESTMENT SCORING")
print("=" * 60)

# Initialize score
score = 0
max_score = 100
reasons = []
concerns = []

# 1. Payback period score (max 40 points)
if payback_years < 6:
    score += 40
    reasons.append("EXCELLENT payback: Less than 6 years")
elif payback_years < 8:
    score += 35
    reasons.append("VERY GOOD payback: 6-8 years")
elif payback_years < 10:
    score += 30
    reasons.append("GOOD payback: 8-10 years")
elif payback_years < 12:
    score += 20
    reasons.append("ACCEPTABLE payback: 10-12 years")
elif payback_years < 15:
    score += 10
    reasons.append("MEDIOCRE payback: 12-15 years")
else:
    concerns.append("POOR payback: More than 15 years")

# 2. ROI score (max 30 points)
if roi_20_years > 300:
    score += 30
    reasons.append("EXCELLENT ROI: >300%")
elif roi_20_years > 200:
    score += 25
    reasons.append("VERY GOOD ROI: 200-300%")
elif roi_20_years > 150:
    score += 20
    reasons.append("GOOD ROI: 150-200%")
elif roi_20_years > 100:
    score += 15
    reasons.append("ACCEPTABLE ROI: 100-150%")
elif roi_20_years > 50:
    score += 5
    reasons.append("LOW ROI: 50-100%")
else:
    concerns.append("VERY LOW ROI: <50%")

# 3. Coverage score (max 30 points)
if coverage_percentage > 120:
    score += 20
    reasons.append(f"EXCELLENT coverage: {coverage_percentage:.0f}% of consumption")
elif coverage_percentage > 80:
    score += 30
    reasons.append(f"GREAT coverage: {coverage_percentage:.0f}% of consumption")
elif coverage_percentage > 60:
    score += 25
    reasons.append(f"GOOD coverage: {coverage_percentage:.0f}% of consumption")
elif coverage_percentage > 40:
    score += 15
    reasons.append(f"PARTIAL coverage: {coverage_percentage:.0f}% of consumption")
else:
    concerns.append(f"LOW coverage: {coverage_percentage:.0f}% of consumption")

print(f"\nTOTAL SCORE: {score}/{max_score}")
print(f"VERDICT: ", end="")

if score >= 80:
    print("HIGHLY RECOMMENDED")
elif score >= 60:
    print("RECOMMENDED")
elif score >= 40:
    print("CONSIDER WITH CAUTION")
else:
    print("NOT RECOMMENDED")

print("\nSTRENGTHS:")
for reason in reasons:
    print(f"   {reason}")

if concerns:
    print("\nCONCERNS:")
    for concern in concerns:
        print(f"   {concern}")

INVESTMENT SCORING

TOTAL SCORE: 20/100
VERDICT: NOT RECOMMENDED

STRENGTHS:
   EXCELLENT coverage: 124% of consumption

CONCERNS:
   POOR payback: More than 15 years
   VERY LOW ROI: <50%


In [7]:
# Cell 7: FINAL RECOMMENDATION
print("=" * 60)
print("FINAL RECOMMENDATION - TURIN")
print("=" * 60)

# Determine recommendation based on score
if score >= 80:
    print("""
╔══════════════════════════════════════════════════════════╗
║                                                          ║
║                 HIGHLY RECOMMENDED                       ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝

Solar panels in Turin are an EXCELLENT investment!

KEY HIGHLIGHTS:
   • Payback period: {payback_years_display}
   • 20-year profit: €{total_profit_20_years:,.0f}
   • ROI: {roi_20_years:.0f}%
   • Covers {coverage_percentage:.1f}% of your consumption

RECOMMENDATIONS:
   1. Install a {optimal_panel_size} kWp system (optimized for your needs)
   2. Aim for {optimal_self_consumption_pct}% self-consumption
   3. Consider adding battery storage in the future
   4. Get quotes from 3-4 local installers
   5. Apply for Scambio Sul Posto incentive
""".format(payback_years_display=payback_years_display, 
           total_profit_20_years=total_profit_20_years,
           roi_20_years=roi_20_years,
           coverage_percentage=coverage_percentage,
           optimal_panel_size=optimal_panel_size,
           optimal_self_consumption_pct=optimal_self_consumption_pct))

elif score >= 60:
    print("""
╔══════════════════════════════════════════════════════════╗
║                                                          ║
║                    RECOMMENDED                           ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝

Solar panels in Turin are a GOOD investment.

KEY HIGHLIGHTS:
   • Payback period: {payback_years_display}
   • 20-year profit: €{total_profit_20_years:,.0f}
   • ROI: {roi_20_years:.0f}%
   • Covers {coverage_percentage:.1f}% of your consumption

RECOMMENDATIONS:
   1. Install a {optimal_panel_size} kWp system
   2. Focus on maximizing self-consumption
   3. Compare quotes from multiple installers
   4. Apply for available incentives
""".format(payback_years_display=payback_years_display,
           total_profit_20_years=total_profit_20_years,
           roi_20_years=roi_20_years,
           coverage_percentage=coverage_percentage,
           optimal_panel_size=optimal_panel_size))

elif score >= 40:
    print("""
╔══════════════════════════════════════════════════════════╗
║                                                          ║
║               CONSIDER WITH CAUTION                      ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝

Solar panels in Turin may be worthwhile, but review carefully.

KEY METRICS:
   • Payback period: {payback_years_display}
   • 20-year profit: €{total_profit_20_years:,.0f}
   • ROI: {roi_20_years:.0f}%

CONSIDERATIONS:
   1. Verify your actual consumption patterns
   2. Check for local shading issues
   3. Consider waiting for better incentives
   4. Get multiple opinions before committing
""".format(payback_years_display=payback_years_display,
           total_profit_20_years=total_profit_20_years,
           roi_20_years=roi_20_years))

else:
    print("""
╔══════════════════════════════════════════════════════════╗
║                                                          ║
║                  NOT RECOMMENDED                         ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝

Solar panels in Turin are NOT recommended at this time.

CONCERNS:
   • Payback period: {payback_years_display} (too long)
   • 20-year profit: €{total_profit_20_years:,.0f} (too low)
   • ROI: {roi_20_years:.0f}% (below threshold)

ALTERNATIVES:
   1. Wait for electricity prices to increase
   2. Look for better incentive programs
   3. Consider energy efficiency improvements first
   4. Review again in 2-3 years
""".format(payback_years_display=payback_years_display,
           total_profit_20_years=total_profit_20_years,
           roi_20_years=roi_20_years))

FINAL RECOMMENDATION - TURIN

╔══════════════════════════════════════════════════════════╗
║                                                          ║
║                  NOT RECOMMENDED                         ║
║                                                          ║
╚══════════════════════════════════════════════════════════╝

Solar panels in Turin are NOT recommended at this time.

CONCERNS:
   • Payback period: Never pays back (too long)
   • 20-year profit: €-7,383 (too low)
   • ROI: -137% (below threshold)

ALTERNATIVES:
   1. Wait for electricity prices to increase
   2. Look for better incentive programs
   3. Consider energy efficiency improvements first
   4. Review again in 2-3 years



In [9]:
# Cell 8: NEXT STEPS
print("=" * 60)
print("NEXT STEPS")
print("=" * 60)

print("""
ACTION PLAN:

1. VERIFY YOUR DATA
   • Check your actual electricity bills (consumption may differ)
   • Verify roof orientation and shading
   • Confirm local installation costs

2. GET PROFESSIONAL QUOTES
   • Request quotes from 3-4 certified installers
   • Compare prices and equipment quality
   • Ask about warranties (panels: 25 years, inverter: 10-12 years)

3. CHECK INCENTIVES
   • Apply for Scambio Sul Posto (grid exchange)
   • Check for regional incentives in Piedmont
   • Verify tax deduction opportunities

4. OPTIMIZE YOUR SYSTEM
   • Consider {optimal_panel_size} kWp for optimal payback
   • Aim for {optimal_self_consumption_pct}% self-consumption
   • Evaluate battery storage if self-consumption is low

5. MONITOR PERFORMANCE
   • Track production in first year
   • Compare with estimates
   • Schedule annual maintenance
""".format(optimal_panel_size=optimal_panel_size,
           optimal_self_consumption_pct=optimal_self_consumption_pct))

NEXT STEPS

ACTION PLAN:

1. VERIFY YOUR DATA
   • Check your actual electricity bills (consumption may differ)
   • Verify roof orientation and shading
   • Confirm local installation costs

2. GET PROFESSIONAL QUOTES
   • Request quotes from 3-4 certified installers
   • Compare prices and equipment quality
   • Ask about warranties (panels: 25 years, inverter: 10-12 years)

3. CHECK INCENTIVES
   • Apply for Scambio Sul Posto (grid exchange)
   • Check for regional incentives in Piedmont
   • Verify tax deduction opportunities

4. OPTIMIZE YOUR SYSTEM
   • Consider 6.0 kWp for optimal payback
   • Aim for 70.0% self-consumption
   • Evaluate battery storage if self-consumption is low

5. MONITOR PERFORMANCE
   • Track production in first year
   • Compare with estimates
   • Schedule annual maintenance



In [11]:
# SAVE FINAL REPORT
print("=" * 60)
print("SAVING FINAL REPORT")
print("=" * 60)

# Create output directory
output_dir = "notebooks_output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save final recommendations
final_report = {
    'score': score,
    'verdict': 'HIGHLY RECOMMENDED' if score >= 80 else 'RECOMMENDED' if score >= 60 else 'CONSIDER WITH CAUTION' if score >= 40 else 'NOT RECOMMENDED',
    'panel_power': panel_power,
    'optimal_panel_size': optimal_panel_size,
    'payback_years': payback_years,
    'roi_20_years': roi_20_years,
    'total_profit_20_years': total_profit_20_years,
    'coverage_percentage': coverage_percentage,
    'optimal_self_consumption': optimal_self_consumption_pct,
    'estimated_annual_production': estimated_annual_kwh,
    'installation_cost': installation_cost,
    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

# Save as pickle
import pickle
report_file = f"{output_dir}/final_recommendations.pkl"
with open(report_file, 'wb') as f:
    pickle.dump(final_report, f)

print(f"Final report saved to: {report_file}")

# Also save as text for easy reading
txt_file = f"{output_dir}/final_recommendations.txt"
with open(txt_file, 'w') as f:
    f.write("=" * 60 + "\n")
    f.write("SOLAR PV ANALYSIS - FINAL RECOMMENDATIONS\n")
    f.write("=" * 60 + "\n\n")
    f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Location: Turin, Italy\n\n")
    f.write(f"VERDICT: {final_report['verdict']}\n")
    f.write(f"Score: {score}/100\n\n")
    f.write(f"Recommended System: {optimal_panel_size} kWp\n")
    f.write(f"Payback Period: {payback_years_display}\n")
    f.write(f"20-year Profit: €{total_profit_20_years:,.0f}\n")
    f.write(f"ROI: {roi_20_years:.0f}%\n")
    f.write(f"Coverage: {coverage_percentage:.1f}%\n")

print(f"Text report saved to: {txt_file}")

SAVING FINAL REPORT
Final report saved to: notebooks_output/final_recommendations.pkl
Text report saved to: notebooks_output/final_recommendations.txt
